In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
from random import sample
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
import glob
import cv2

#from google.colab import drive

import matplotlib.pyplot as plt
import keras as ks
from keras import layers
from keras.models import Sequential
import tensorflow as tf
from keras.backend import categorical_crossentropy

import os

from keras.applications.inception_v3 import preprocess_input
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import SGD

In [3]:
image_size = (250, 250)
batch_size = 32

 
train_directory = '/content/drive/MyDrive/Colab Notebooks/BT5153 Applied ML Project /5 classes 5k each for train/Training'
val_directory = '/content/drive/MyDrive/Colab Notebooks/BT5153 Applied ML Project /5 classes 5k each for train/Validation'
test_directory = '/content/drive/MyDrive/Colab Notebooks/BT5153 Applied ML Project /5 classes 5k each for train/Testing'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(val_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)

train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)
test_ds = test_ds.prefetch(buffer_size=32)

Found 25000 files belonging to 5 classes.
Found 27447 files belonging to 5 classes.
Found 34281 files belonging to 5 classes.


In [4]:
input_shape = (250, 250)
nclass = 5

# create the base pre-trained model 
base_model = InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape= (input_shape+(3,)))

# to freeze the base model 
# https://keras.io/guides/transfer_learning/
#base_model.trainable = False

# to add layers to the model 
add_model = Sequential()
add_model.add(base_model)
add_model.add(layers.Rescaling(1.0/255))
add_model.add(layers.Conv2D(32, 3, strides=2, padding='same'))
add_model.add(layers.BatchNormalization())
add_model.add(layers.Activation('relu'))
add_model.add(GlobalAveragePooling2D()) # to add a global spatial average pooling layer 
add_model.add(Dropout(0.5))
add_model.add(Dense(nclass, activation='softmax')) #there are 5 classes 

# compile the model (should be done *after* setting layers to non-trainable)
model = add_model
model.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 rescaling (Rescaling)       (None, 6, 6, 2048)        0         
                                                                 
 conv2d_94 (Conv2D)          (None, 3, 3, 32)          589856    
                                                                 
 batch_normalization_94 (Bat  (None, 3, 3, 32)         128       
 chNormalization)                                                
                                                                 
 activation_94 (Activation)  (None, 3, 3, 32)          0         
                                                                 
 global_average_pooling2d (G  (None, 32)               0         
 lobalAveragePooling2D)                                 

In [5]:
file_path="/content/drive/MyDrive/Colab Notebooks/5153 weights/inceptionv3_V2_2_5C5K_{epoch}.h5"
epochs = 1000

checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=10)

callbacks_list = [checkpoint, early] #early

history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks_list)

Epoch 1/1000
782/782 [==============================] - ETA: 0s - loss: 0.8807 - accuracy: 0.6966
Epoch 1: val_loss improved from inf to 1.64368, saving model to /content/drive/MyDrive/Colab Notebooks/5153 weights/inceptionv3_V2_2_5C5K_1.h5
782/782 [==============================] - 341s 417ms/step - loss: 0.8807 - accuracy: 0.6966 - val_loss: 1.6437 - val_accuracy: 0.4457
Epoch 2/1000
782/782 [==============================] - ETA: 0s - loss: 0.6258 - accuracy: 0.7969
Epoch 2: val_loss improved from 1.64368 to 0.78223, saving model to /content/drive/MyDrive/Colab Notebooks/5153 weights/inceptionv3_V2_2_5C5K_2.h5
782/782 [==============================] - 318s 406ms/step - loss: 0.6258 - accuracy: 0.7969 - val_loss: 0.7822 - val_accuracy: 0.7083
Epoch 3/1000
782/782 [==============================] - ETA: 0s - loss: 0.5273 - accuracy: 0.8331
Epoch 3: val_loss did not improve from 0.78223
782/782 [==============================] - 317s 405ms/step - loss: 0.5273 - accuracy: 0.8331 - val_

In [7]:
from sklearn.metrics import accuracy_score

In [8]:
##load model at epoch 5 --> best weight (alternatively, set restore_best_weights = True when fitt)
new_model = ks.models.load_model('/content/drive/MyDrive/Colab Notebooks/5153 weights/inceptionv3_V2_2_5C5K_5.h5')


In [9]:
true = []
pred = []
batches = int(np.ceil(34281/32))
i = 1
for image, label in test_ds:
  pred = np.concatenate([pred, 
                         np.argmax(new_model.predict(image), axis=-1)]) #use new_model
  true = np.concatenate([true,
                         np.argmax(label.numpy(), axis=-1)])
  print('{}/{}'.format(i, batches))
  i += 1

1/1072
2/1072
3/1072
4/1072
5/1072
6/1072
7/1072
8/1072
9/1072
10/1072
11/1072
12/1072
13/1072
14/1072
15/1072
16/1072
17/1072
18/1072
19/1072
20/1072
21/1072
22/1072
23/1072
24/1072
25/1072
26/1072
27/1072
28/1072
29/1072
30/1072
31/1072
32/1072
33/1072
34/1072
35/1072
36/1072
37/1072
38/1072
39/1072
40/1072
41/1072
42/1072
43/1072
44/1072
45/1072
46/1072
47/1072
48/1072
49/1072
50/1072
51/1072
52/1072
53/1072
54/1072
55/1072
56/1072
57/1072
58/1072
59/1072
60/1072
61/1072
62/1072
63/1072
64/1072
65/1072
66/1072
67/1072
68/1072
69/1072
70/1072
71/1072
72/1072
73/1072
74/1072
75/1072
76/1072
77/1072
78/1072
79/1072
80/1072
81/1072
82/1072
83/1072
84/1072
85/1072
86/1072
87/1072
88/1072
89/1072
90/1072
91/1072
92/1072
93/1072
94/1072
95/1072
96/1072
97/1072
98/1072
99/1072
100/1072
101/1072
102/1072
103/1072
104/1072
105/1072
106/1072
107/1072
108/1072
109/1072
110/1072
111/1072
112/1072
113/1072
114/1072
115/1072
116/1072
117/1072
118/1072
119/1072
120/1072
121/1072
122/1072
123/1072
1

In [10]:
accuracy_score(true, pred)

0.7425687698725242

In [11]:
model_results = pd.DataFrame({'true':true,
                              'pred':pred})

model_results.to_csv('/content/drive/MyDrive/Colab Notebooks/5153 weights/inceptionv3_V2_2_test_pred.csv')